# $\alpha$=0.1

In [4]:
import torch
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet50_Weights
import numpy as np
import torchvision.transforms as transforms        
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from src.temperature_scaling import ModelWithTemperature
from src.saps import split_data_set, saps_scores, saps_classification, eval_aps

#  Reprocess
data_transform = transforms.Compose([
    transforms.CenterCrop(256),
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

sorted_val_path = "D:\\Download\\ImageNet-1K\\Validation_Set\\sorted_ImageNet_val"
dataset = ImageFolder(root=sorted_val_path, transform=data_transform)

# load pre-trained model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1).to(device)

# Temperature Scaling
subset_size = len(dataset) // 10
indices = np.random.choice(len(dataset), subset_size, replace=False)
subset_dataset = Subset(dataset, indices)
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False, num_workers=2)
model = ModelWithTemperature(model, temperature = 1.0).to(device)
model.set_temperature(train_loader)

model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1
# Hyperparameter
lambda_ = 0.15

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("\n")
print("SAPS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, q_hat, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Before temperature - NLL: 1.140, ECE: 0.026
Optimal temperature: 0.983
After temperature - NLL: 1.139, ECE: 0.025


SAPS Classification, Start!

Running experiment 1/10...
q_hat = 1.068893873691559
Average Prediction Set Size After APS in runs 1: 3.06352
Average Coverage Rate in runs 1: 90.11%

Running experiment 2/10...
q_hat = 1.0559655308723455
Average Prediction Set Size After APS in runs 2: 2.96352
Average Coverage Rate in runs 2: 89.75%

Running experiment 3/10...
q_hat = 1.0796450495719911
Average Prediction Set Size After APS in runs 3: 3.12436
Average Coverage Rate in runs 3: 90.46%

Running experiment 4/10...
q_hat = 1.0584881663322452
Average Prediction Set Size After APS in runs 4: 2.99404
Average Coverage Rate in runs 4: 89.80%

Running experiment 5/10...
q_hat = 1.053405380249025
Average Prediction Set Size After APS in runs 5: 2.95248
Average Coverage Rate in runs 5: 89.56%

Running experiment 6/10...
q_hat = 1.0803406476974493
Average Prediction Set Size After APS in ru

# $\alpha$=0.2

In [3]:
# error rate
alpha = 0.2
# Hyperparameter
lambda_ = 0.1

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("\n")
print("SAPS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    q_hat = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, q_hat, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")



SAPS Classification, Start!

Running experiment 1/10...
q_hat = 0.8278432846069337
Average Prediction Set Size After APS in runs 1: 2.41748
Average Coverage Rate in runs 1: 80.25%

Running experiment 2/10...
q_hat = 0.8240817308425904
Average Prediction Set Size After APS in runs 2: 2.37836
Average Coverage Rate in runs 2: 79.43%

Running experiment 3/10...
q_hat = 0.8358584165573121
Average Prediction Set Size After APS in runs 3: 2.44236
Average Coverage Rate in runs 3: 81.05%

Running experiment 4/10...
q_hat = 0.8215206742286683
Average Prediction Set Size After APS in runs 4: 2.37808
Average Coverage Rate in runs 4: 79.33%

Running experiment 5/10...
q_hat = 0.8269466042518616
Average Prediction Set Size After APS in runs 5: 2.40844
Average Coverage Rate in runs 5: 79.80%

Running experiment 6/10...
q_hat = 0.8299161553382874
Average Prediction Set Size After APS in runs 6: 2.43592
Average Coverage Rate in runs 6: 80.48%

Running experiment 7/10...
q_hat = 0.8268536329269409
Ave

# $\alpha$=0.05

In [2]:
# error rate
alpha = 0.05
# Hyperparameter
lambda_ = 0.05

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("\n")
print("SAPS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    q_hat = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, q_hat, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")



SAPS Classification, Start!

Running experiment 1/10...
q_hat = 1.0883428096771237
Average Prediction Set Size After APS in runs 1: 7.68516
Average Coverage Rate in runs 1: 94.96%

Running experiment 2/10...
q_hat = 1.0834756791591644
Average Prediction Set Size After APS in runs 2: 7.54936
Average Coverage Rate in runs 2: 94.88%

Running experiment 3/10...
q_hat = 1.0913856208324433
Average Prediction Set Size After APS in runs 3: 7.72528
Average Coverage Rate in runs 3: 95.03%

Running experiment 4/10...
q_hat = 1.0899467229843138
Average Prediction Set Size After APS in runs 4: 7.71032
Average Coverage Rate in runs 4: 95.02%

Running experiment 5/10...
q_hat = 1.0781174302101135
Average Prediction Set Size After APS in runs 5: 7.47272
Average Coverage Rate in runs 5: 94.78%

Running experiment 6/10...
q_hat = 1.1045667886734007
Average Prediction Set Size After APS in runs 6: 8.03472
Average Coverage Rate in runs 6: 95.22%

Running experiment 7/10...
q_hat = 1.092527878284454
Aver

## Result  
$\alpha$=0.1  
From the above test, following results can be collected :
- Final Average Prediction Set Size: 3.04
- Final Average Coverage: 90.01%  

$\alpha$=0.2  
From the above test, following results can be collected :
- Final Average Prediction Set Size: 3.04
- Final Average Coverage: 90.01%  

$\alpha$=0.05  
From the above test, following results can be collected :
- Final Average Prediction Set Size: 2.41
- Final Average Coverage: 80.06%